# Good practices

In [ ]:
#import
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score

%matplotlib inline
import matplotlib.pyplot as plt

#matplotlib style options
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 8)

In [ ]:
#format
df['datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")

#type category
de['model'] = df['model'].astype('category')
df = pd.get_dummies(df)

#merge
features = df1.merge(df2, on=['datetime', 'machineID'], how='left')

#filna
df = df.fillna(method='bfill', limit=7) # fill backward up to limit entries 
df = df.fillna('none')

#category to binary:
df['comp1_fail'] = (df['failure'] == 'comp1').astype(int)

In [ ]:
#evaluate
def evaluate(y_true, y_pred, print_cm=False):
    # calculate and display confusion matrix
    labels = np.unique(y_true)
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    if print_cm:
        print('Confusion matrix\n- x-axis is true labels (none, comp1, etc.)\n- y-axis is predicted labels')
        print(cm)

    # calculate precision, recall, and F1 score
    accuracy = float(np.trace(cm)) / np.sum(cm)
    precision = precision_score(y_true, y_pred, average=None, labels=labels)[1]
    recall = recall_score(y_true, y_pred, average=None, labels=labels)[1]
    f1 = 2 * precision * recall / (precision + recall)
    print("accuracy:", accuracy)
    print("precision:", precision)
    print("recall:", recall)
    print("f1 score:", f1)

In [ ]:
#shuffle data
from sklearn.utils import shuffle
shuffled_features = shuffle(df)

# standardize data
X_mean = X_train_shuffled.mean(axis=0)
X_std = X_train_shuffled.std(axis=0)
X_train_shuffled = (X_train_shuffled - X_mean) / X_std
X_val_shuffled = (X_val_shuffled - X_mean) / X_std
X_test_shuffled = (X_test_shuffled - X_mean) / X_std

In [ ]:
#models
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

#or with l2 regularization
model = LogisticRegression(C=0.1, random_state=42)
model.fit(X_train, y_train)

#or l1 regularization # shows irrelevant features
model = LogisticRegression(penalty='l1', C=0.1, random_state=42)
model.fit(X_train, y_train)
# find irrelevant features
for i in range(len(features_to_use)):
    if model.coef_[0][i] == 0:
        print("Feature %s is irrelevant" % (features_to_use[i],))
        
####

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print("- Train set results:")
evaluate(y_train, y_pred_train)
print("- Test set results:")
evaluate(y_test, y_pred_test)


In [ ]:
#NN keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

# function to fit nnet
def fit_nnet(X_train, y_train, X_val, y_val, num_epochs=15, batch_size=2048):
    # define the keras model
    model = Sequential()
    model.add(Dense(150, input_dim=30, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(30, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(1, activation='sigmoid'))

    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # fit the keras model on the dataset
    history = model.fit(X_train, y_train, validation_data=(X_val,y_val),
                        epochs=num_epochs, batch_size=batch_size, verbose=2)
    return model, history

# function to evaluate nnet on some data
def eval_nnet(model, X_new, y_true):
    # evaluate the keras model
    y_pred = model.predict(X_new)
    y_pred = (y_pred > 0.5).astype(int)

    # evaluate predictions
    evaluate(y_true, y_pred)
    

fitted_model, history = fit_nnet(X_train, y_train, X_val, y_val, num_epochs=10, batch_size=2048)

print("- Train set results:")
eval_nnet(fitted_model, X_train, y_train)
print("- Test set results:")
eval_nnet(fitted_model, X_test, y_test)

#plot gistory
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(["train loss", "val loss"])
plt.show()

# Time Series descriptive

In [1]:
# Add columns with year, month, and weekday name
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Weekday Name'] = df.index.weekday_name

#select specific day
df.loc['2017-08-10']
df.loc['2014-01-20':'2014-01-22']

#drop na
df.dropna()

# seaborn styling for our plots, and let’s adjust the 
# default figure size to an appropriate shape for time series plots.
import seaborn as sns
sns.set(rc={'figure.figsize':(14, 5)})

#plot selected features
cols_plot = ['Consumption', 'Solar', 'Wind']
axes = df[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(14, 11), subplots=True)
for ax in axes:
    ax.set_ylabel('Daily Totals (GWh)')

#loc certain time and plot 1 feature
ax = df.loc['2017-01':'2017-02', 'Consumption'].plot()
ax.set_ylabel('Daily Consumption (GWh)');

#weekends
weekends=df.loc['2017-01':'2017-02', 'Consumption'].index.weekday>=5
colors=['blue' if x else 'red' for x in weekends]

#formatted plot for weeks
import matplotlib.dates as mdates

fig, ax = plt.subplots()
ax.plot(opsd_daily.loc['2017-01':'2017-02', 'Consumption'])
ax.scatter(opsd_daily.loc['2017-01':'2017-02'].index, opsd_daily.loc['2017-01':'2017-02', 'Consumption'], marker='o', linestyle='-', c=colors)
ax.set_ylabel('Daily Consumption (GWh)')
ax.set_title('Jan-Feb 2017 Electricity Consumption')

# To better visualize the weekly seasonality in electricity consumption 
# we add vertical gridlines on a weekly time scale

# Set x-axis major ticks to weekly interval, on Mondays
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY))
# Format x-tick labels as 3-letter month name and day number
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'));



NameError: name 'df' is not defined

In [ ]:
#BOXPLOTS
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(['Consumption', 'Solar', 'Wind'], axes):
    sns.boxplot(data=df, x='Month', y=name, ax=ax)#need to add month collumn before
    ax.set_ylabel('GWh')
    ax.set_title(name)
    # Remove the automatic x-axis label from all but the bottom subplot
    if ax != axes[-1]:
        ax.set_xlabel('')
        
#for weekdays
sns.boxplot(data=df, x='Weekday Name', y='Consumption');

In [ ]:
#resampling

# Specify the data columns we want to include (i.e. exclude Year, Month, Weekday Name)
data_columns = ['Consumption', 'Wind', 'Solar', 'Wind+Solar', 'gap', 'diff']
# Resample to weekly frequency, aggregating with mean
opsd_weekly_mean = df[data_columns].resample('W').mean()
opsd_weekly_mean[:3]

#rolling windows
opsd_7d = df[data_columns].rolling(7, center=True).mean()
opsd_365d = df[data_columns].rolling(window=365, center=True, min_periods=360).mean()


In [ ]:
#autocorrelation
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(opsd_daily['Consumption'])
plot_pacf(opsd_daily['Consumption']);
#looks at the correlation every lag of X

# Time Series applied

In [ ]:
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')
import random
import pandas as pd
import numpy as np  
from sklearn.model_selection import TimeSeriesSplit # you have everything done for you


#evaluate results
from sklearn.metrics import r2_score, mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#plot original and predicted
def plotprediction(series, pred_series, labels=["original", "predicted"], x_axis=None, plot_intervals=False, scale=1.96, plot_anomalies=False, title="prediction"):

    plt.figure(figsize=(15,5))
    plt.title(title)
    if x_axis is None:
        x_axis=series.index
    
    plt.plot(x_axis, pred_series, "g", label=labels[1])
    plt.plot(x_axis, series, label=labels[0])
    

    # Plot confidence intervals for smoothed values

    plt.legend(loc="upper left")
    plt.grid(True)
    
def timeseries_train_test_split_indexes(ts, test_size):
    """
        Gets the two vectors with indexes for the train test split (first vector is train observations, 
        second vector is test observations).
    """
    
    # get the index after which test set starts
    split_time = int(len(ts)*(1-test_size))
    
    test_index=ts.iloc[split_time:].index[0]
    
    return ts.index<test_index, ts.index>=test_index

In [ ]:

#train test split
ads_train_ix, ads_test_ix=timeseries_train_test_split_indexes(df, 0.3)

#look at basic lag of 1
df['copy_pred']=[0]+df['Ads'].to_list()[:-1]
#plot
plotprediction(df[ads_test_ix]['Ads'], df[ads_test_ix]['copy_pred'], labels=['observed', 'copy based prediction'], title="Naive method 1")

#stats
print_stats(ads[ads_test_ix]['Ads'], ads[ads_test_ix]['copy_pred'], "copy method")

#apply rolling and print
plotprediction(df['GEMS_GEMS_SPENT'], df['GEMS_GEMS_SPENT'].rolling(4).mean(), x_axis=currency.index,labels=['observed', 'moving average smoothing (p=24)'], title="Smoothing")


#look at autocorrelation
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(ads_data['Ads'])
plot_pacf(ads_data['Ads']);
####look at PACF for AR (count ones above interval)
####look at ACF for MA (countones clearly above interval)


#basic linear regression
from sklearn.linear_model import LinearRegression

y = ads_data.Ads
X = ads_data.drop(['Ads'], axis=1)

X_train, X_test, y_train, y_test = X[ads_train_ix], X[ads_test_ix], y[ads_train_ix], y[ads_test_ix]

X_train=X_train[LAGS:]  #because the first LAGS observations will have NaNs
y_train=y_train[LAGS:]

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred=lr.predict(X_test)

plotprediction(y_test, y_pred)


#check result
print_stats(y_test, y_pred, "AR (3)")


# MA example
from statsmodels.tsa.arima_model import ARMA, ARIMA


##ARIMA
# run adf to se if integration is necessary
from statsmodels.tsa.stattools import adfuller
result = adfuller(ads['Ads'])
print('ADF Statistic: %f' % result[0]) #the more negative the higher the chance of stationary (1% should be higher)
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
   print('\t%s: %.3f' % (key, value))
# https://machinelearningmastery.com/time-series-data-stationary-python/

preds=[]
for l in ads[ads_test_ix]['Ads'].index:
    data=ads['Ads'][ads.index<l]
    model=ARIMA(data, order=(3, 0, 2)) #Notice that now we're using the ARIMA object
    arima=model.fit()
    
    preds.append(arima.forecast()[0])
    
y_pred=np.array(preds).T[0]
print(arima.summary())

#plot
y_test=ads[ads_test_ix]['Ads']
plotprediction(y_test, y_pred)

print_stats(y_test, y_pred)

# Clustering